### 1. Create an index to tune this query:
`select * from shopify.apps a join shopify.reviews r on a.id = r.app_id where a.rating >= 5.0 and r.author = 'Funky Moose Records';`
#### Test your index using `explain` and time the query using `\timing` in psql

In [1]:
!psql -c "explain select * from shopify.apps a join shopify.reviews r on a.id = r.app_id where a.rating >= 5.0 and r.author = 'Funky Moose Records'"


                                   QUERY PLAN                                   
--------------------------------------------------------------------------------
 Nested Loop  (cost=0.28..2970.83 rows=1 width=374)
   ->  Seq Scan on reviews r  (cost=0.00..2962.51 rows=1 width=60)
         Filter: ((author)::text = 'Funky Moose Records'::text)
   ->  Index Scan using apps_pkey on apps a  (cost=0.28..8.30 rows=1 width=314)
         Index Cond: (id = r.app_id)
         Filter: (rating >= '5'::double precision)
(6 rows)



In [2]:
!psql -c "create index reviews_author_idx on shopify.reviews(author)"

CREATE INDEX


In [3]:
!psql -c "explain select * from shopify.apps a join shopify.reviews r on a.id = r.app_id where a.rating >= 5.0 and r.author = 'Funky Moose Records'"

                                        QUERY PLAN                                         
-------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.70..16.75 rows=1 width=374)
   ->  Index Scan using reviews_author_idx on reviews r  (cost=0.42..8.44 rows=1 width=60)
         Index Cond: ((author)::text = 'Funky Moose Records'::text)
   ->  Index Scan using apps_pkey on apps a  (cost=0.28..8.30 rows=1 width=314)
         Index Cond: (id = r.app_id)
         Filter: (rating >= '5'::double precision)
(6 rows)



#### Prompt 1: Choose a query you wrote for Project 3. Try to find a relevant index to tune your query. Provide the SQL for your query, the create index statement, and the explain plan.